In [1]:
# import os

# # Clone the repository
# !git clone https://github.com/FangyunWei/SLRT.git
# os.chdir("SLRT")
# print("Current Working Directory:", os.getcwd())

In [2]:
# !pip install decord

In [3]:
from turtle import forward
import warnings
import pickle
from collections import defaultdict
from modelling.model import build_model
from utils.optimizer import build_optimizer, build_scheduler
warnings.filterwarnings("ignore")
import argparse
import numpy as np
import os, sys
import shutil
import time
import queue
sys.path.append(os.getcwd())#slt dir
import torch
from torch.nn.parallel import DistributedDataParallel as DDP, distributed
import torch.nn.functional as F
from torch import Tensor
from torch.utils.tensorboard import SummaryWriter
from utils.misc import (
    load_config,
    log_cfg,
    load_checkpoint,
    make_writer,
    set_seed,
    symlink_update,
    is_main_process, init_DDP, move_to_device,
    neq_load_customized,
    synchronize,
)
from utils.metrics import compute_accuracy
from dataset.Dataloader import build_dataloader
from dataset.Dataset import build_dataset
from utils.progressbar import ProgressBar
from copy import deepcopy
import datetime

In [4]:
with open("pickle_data/test.pkl", "rb") as f:
    test_split = pickle.load(f)

In [5]:
vocab_list = sorted({item['label'] for item in test_split})
cls_num = len({item['label'] for item in test_split})

In [6]:
cls_num = 100

In [7]:
with open("pickle_data/wlasl_word_embeddings.pkl", "rb") as f:
    word_emb_tab = pickle.load(f)

# 2. Determine embedding dimension (check the first item in the dict)
first_key = list(word_emb_tab.keys())[0]
first_val = word_emb_tab[first_key]
embed_dim = len(first_val)

# 3. Create the Tensor Matrix
# Initialize with zeros or random noise
embedding_tensor = torch.zeros((len(vocab_list), embed_dim))

print(f"Building embedding matrix for {len(vocab_list)} classes...")

# 4. Fill the matrix ensuring the order matches the class IDs
for i, word in enumerate(vocab_list):
    # Strip whitespace just in case
    word_clean = word.strip() if isinstance(word, str) else word
    
    if word_clean in word_emb_tab:
        # Convert list/numpy array to torch tensor
        vector = torch.tensor(word_emb_tab[word_clean])
        embedding_tensor[i] = vector
    else:
        # Optional: Print warning if a word in your dataset lacks an embedding
        # print(f"Warning: '{word}' not found in embedding dictionary.")
        pass

Building embedding matrix for 1999 classes...


In [8]:
parser = argparse.ArgumentParser("SLT baseline Testing")
parser.add_argument("--config", default="configs/nla_slr_wlasl_100.yaml", type=str, help="Training configuration file (yaml).")
parser.add_argument("--save_subdir", default='prediction', type=str)
parser.add_argument('--ckpt_name', default='best.ckpt', type=str)
parser.add_argument('--eval_setting', default='origin', type=str)
args = parser.parse_args([])
cfg = load_config(args.config)

# Configure the environment variables for a single process (the notebook)
os.environ['MASTER_ADDR'] = 'localhost'
os.environ['MASTER_PORT'] = '12355' # Any free port
os.environ['RANK'] = '0'
os.environ['WORLD_SIZE'] = '1'

# NOTE: Since you are setting device to 'cpu' in line 12, 
# you MUST use 'gloo' instead of 'nccl'. 'nccl' only works on CUDA GPUs.
backend = "nccl" 

torch.distributed.init_process_group(
    backend=backend, 
    init_method="env://", 
    timeout=datetime.timedelta(0, 3600)
)

# Set LOCAL_RANK and WORLD_SIZE environment variables for single-process execution
cfg['local_rank'] = 0; cfg['world_size']=1; cfg['device'] = torch.device('cuda:0')

set_seed(seed=cfg["training"].get("random_seed", 42))
model_dir = cfg['training']['model_dir']
os.makedirs(model_dir, exist_ok=True)


model = build_model(cfg, cls_num, word_emb_tab=embedding_tensor)
model = torch.nn.SyncBatchNorm.convert_sync_batchnorm(model)
#load model
load_model_path = os.path.join('checkpoint',args.ckpt_name)
if os.path.isfile(load_model_path):
    state_dict = torch.load(load_model_path, map_location='cuda')
    neq_load_customized(model, state_dict['model_state'], verbose=True)
    epoch, global_step = state_dict.get('epoch',0), state_dict.get('global_step',0)
    print('Load model ckpt from '+load_model_path)
else:
    print(f'{load_model_path} does not exist')
    epoch, global_step = 0, 0

model = DDP(model,
        device_ids=[cfg['local_rank']],
        output_device=cfg['local_rank'],
        find_unused_parameters=True)

Overwrite cfg.model.RecognitionNetwork.keypoint_s3d.in_channel -> 63
Load pretrained S3D backbone from checkpoint
['checkpoint/S3D_kinetics400.pt']

=======Check Weights Loading======
Weights not used from pretrained file:
backbone.fc.0.weight
backbone.fc.0.bias
---------------------------
Weights not loaded into new model:

Load pretrained S3D backbone from checkpoint
['checkpoint/S3D_kinetics400.pt']

=======Check Weights Loading======
Weights not used from pretrained file:
backbone.base.0.conv_s.weight
backbone.fc.0.weight
backbone.fc.0.bias
---------------------------
Weights not loaded into new model:
backbone.base.0.conv_s.weight shape mis-matched, not loaded

Load pretrained S3D backbone from checkpoint
['checkpoint/S3D_kinetics400.pt']

=======Check Weights Loading======
Weights not used from pretrained file:
backbone.fc.0.weight
backbone.fc.0.bias
---------------------------
Weights not loaded into new model:

Load pretrained S3D backbone from checkpoint
['checkpoint/S3D_kinet